# Fine-tune Tutorial ruBERT-tiny2

Загружаем библиотеки

In [6]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModel
from datasets import Dataset
import evaluate

Загружаем отобранные предложения. Предложения отбираются в скрипте preperror.py

In [9]:
with open('FullDN.json', encoding="utf-8") as f:
    sentences = pd.read_json(f, orient='records')
sentences.tail()

,text,label
10621,Предупреждение действовало до семи утра.По инф...,0
10622,Украина заняла шестоеместо.\n,0
10623,"""Если есть паритет и достоинство, Тайвань очен...",0
10624,"""В соответствие с докладной запиской директора...",0
10625,"""В последующем все виды работ были оплачены, н...",0


In [10]:
raw_ds = Dataset.from_pandas(sentences, preserve_index=False)
raw_ds = raw_ds.class_encode_column('label')
raw_ds.features

Stringifying the column:   0%|          | 0/10626 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10626 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['0', '1'], id=None)}

Datasets:

In [11]:
raw_ds = raw_ds.train_test_split(test_size=0.2, shuffle=True)

In [19]:
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
# model = AutoModelForMaskedLM.from_pretrained("cointegrated/rubert-tiny2", num_labels=2)
# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=2, is_decoder=True)
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def tokenize_function(data):
    return tokenizer(data["text"], truncation=True)
tokenized_ds = raw_ds.map(tokenize_function, batched=True)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds.set_format("torch")
tokenized_ds.column_names

Map:   0%|          | 0/8500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2126 [00:00<?, ? examples/s]

{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'test': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_ds["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_ds["test"], batch_size=8, collate_fn=data_collator
)

In [23]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 110]),
 'token_type_ids': torch.Size([8, 110]),
 'attention_mask': torch.Size([8, 110])}

In [24]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6823, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [26]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/home/vboxuser/dev/bertnews/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

3189


In [31]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

In [32]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/3189 [00:00<?, ?it/s]

In [33]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.7365945437441204, 'f1': 0.759656652360515}

In [75]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [76]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

In [77]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [78]:
trainer.train()

  0%|          | 0/1230 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

{'eval_loss': 0.09038888663053513, 'eval_accuracy': 0.9841269841269841, 'eval_f1': 0.9919999999999999, 'eval_runtime': 29.1574, 'eval_samples_per_second': 28.089, 'eval_steps_per_second': 3.533, 'epoch': 1.0}


Checkpoint destination directory test-trainer/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.0718, 'learning_rate': 2.9674796747967482e-05, 'epoch': 1.22}


...

Checkpoint destination directory test-trainer/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 0.0824, 'learning_rate': 9.34959349593496e-06, 'epoch': 2.44}


  0%|          | 0/103 [00:00<?, ?it/s]

{'eval_loss': 0.08639433979988098, 'eval_accuracy': 0.9841269841269841, 'eval_f1': 0.9919999999999999, 'eval_runtime': 33.948, 'eval_samples_per_second': 24.125, 'eval_steps_per_second': 3.034, 'epoch': 3.0}
{'train_runtime': 1375.8338, 'train_samples_per_second': 7.139, 'train_steps_per_second': 0.894, 'train_loss': 0.07505875641737526, 'epoch': 3.0}


TrainOutput(global_step=1230, training_loss=0.07505875641737526, metrics={'train_runtime': 1375.8338, 'train_samples_per_second': 7.139, 'train_steps_per_second': 0.894, 'train_loss': 0.07505875641737526, 'epoch': 3.0})